# Lab 2: Language Models

Student: John Wu

__Summary__

In [1]:
import charlm as clm
import math

## (a) Simple Character LM

train language model

In [2]:
mdl = clm.train_char_lm('./data/subtitles.txt', order=4)

Continuations for `itte`

In [3]:
clm.print_probs(mdl, 'itte')

[('n', 0.30644454186760195),
 ('r', 0.28233231039017975),
 ('d', 0.2770714598860149),
 ('e', 0.11047786058746165),
 (' ', 0.007452871547566856),
 ('s', 0.0070144673388864535),
 ('.', 0.003068829460762823),
 ('l', 0.0021920210434020165),
 (',', 0.0017536168347216134),
 ('\n', 0.00043840420868040335),
 ('!', 0.00043840420868040335),
 ("'", 0.00043840420868040335),
 ('k', 0.00043840420868040335),
 ('t', 0.00043840420868040335)]


Continuations for `supe`

In [4]:
clm.print_probs(mdl, 'supe')

[('r', 0.9992144540455616), ('s', 0.0007855459544383347)]


Continuations for `ther`

In [5]:
clm.print_probs(mdl, 'ther')

[('e', 0.41045090602612727),
 (' ', 0.3175606525796159),
 ('.', 0.10865089398591295),
 (',', 0.038119318523869725),
 ('s', 0.03071458671964361),
 ("'", 0.022394798627415568),
 ('?', 0.02027572090783216),
 ('i', 0.008295707663596412),
 ('!', 0.007489013304436819),
 ('w', 0.00744085244717356),
 ('f', 0.006983324303172596),
 ('a', 0.005911745229065077),
 ('-', 0.003383300222743965),
 ('\n', 0.0029498525073746312),
 ('n', 0.002925772078743002),
 ('m', 0.0018662332189512973),
 ('l', 0.0010354584311600746),
 ('h', 0.0009752573595810005),
 ('"', 0.0006260911444223707),
 ('t', 0.00046956835831677806),
 (':', 0.00025284450063211124),
 ('b', 0.00018060321473722233),
 ('y', 0.0001685630004214075),
 (']', 0.00015652278610559268),
 (')', 0.0001083619288423334),
 ('o', 9.632171452651857e-05),
 ('/', 6.0201071579074104e-05),
 (';', 6.0201071579074104e-05),
 ('g', 6.0201071579074104e-05),
 ('´', 6.0201071579074104e-05),
 ('j', 4.816085726325929e-05),
 ('c', 3.6120642947444464e-05),
 ('\xa0', 3.6120642

Random Sentences

In [6]:
print(clm.generate_text(mdl, 4, 80) + '\n')
print(clm.generate_text(mdl, 4, 80) + '\n')
print(clm.generate_text(mdl, 4, 80) + '\n')

It is guard, Jacob?
Chambal ran operate?
But Nikki and check the house it is tha

No. Hello
Must've a hits he idea took by across-dried even then?
I doned.
- Just

We'll have time waltzers... and your bettest because of the on concluded kill ho



## (b) Calculate Perplexity

In [7]:
sent = 'The car eats forks and knives'
print(clm.perplexity(sent, mdl, 4))

7.160948621456589


In [8]:
sent = 'The yob eats forks and knives'
print(clm.perplexity(sent, mdl, 4))

inf


In [9]:
sent = 'The student loves homework'
print(clm.perplexity(sent, mdl, 4))

4.606972940490917


In [10]:
sent = 'It is raining in London'
print(clm.perplexity(sent, mdl, 4))

3.7112360009044503


In [11]:
sent = 'asdfjkl; qwerty'
print(clm.perplexity(sent, mdl, 4))

inf


## (c) Naive smoothing

In [12]:
sent = 'The car eats forks and knives'
print(clm.smoothed_perplexity(sent, mdl, 4))

7.160948621456589


In [13]:
sent = 'The yob eats forks and knives'
print(clm.smoothed_perplexity(sent, mdl, 4))

49.00672281265043


In [14]:
sent = 'The student loves homework'
print(clm.smoothed_perplexity(sent, mdl, 4))

4.606972940490917


In [15]:
sent = 'It is raining in London'
print(clm.smoothed_perplexity(sent, mdl, 4))

3.7112360009044503


In [16]:
sent = 'asdfjkl; qwerty'
print(clm.smoothed_perplexity(sent, mdl, 4))

2344635.520980603


## (d) Language identification

list of languages

In [77]:
LANGUAGES = ['fr', 'de', 'da', 'en', 'it', 'nl']
filePat = './data/%s.train.txt'

with open('./data/test.txt', 'r') as f:
    testData = [l.split('\t') for l in f.read().splitlines()]

def trainCharLM(order=0):
    lms = dict()
    for l in LANGUAGES:
        lms[l] = clm.train_char_lm(filePat%l, order)
    return lms

def langID(txt, lms, order):
    scores = dict()
    for l in LANGUAGES:
        scores[l] = clm.smoothed_perplexity(txt, lms[l], order)
    return min(scores, key=scores.get)

def assessLM(lms, order):
    accuracy = {l:[0,0] for l in LANGUAGES}
    for lang, txt in testData:
        accuracy[lang][1] += 1
        pred = langID(txt, lms, order)
        if pred == lang:
            accuracy[lang][0] += 1
    
    for lang,(num,denom) in accuracy.items():
        print('%s: %u correct out of %u -- %.1f%%' % 
              (lang,num,denom,num/denom*100))


__Unigram models__

In [78]:
u_lms = trainCharLM(0)
assessLM(u_lms, 0)

fr: 194 correct out of 200 -- 97.0%
de: 178 correct out of 200 -- 89.0%
da: 187 correct out of 200 -- 93.5%
en: 187 correct out of 200 -- 93.5%
it: 188 correct out of 200 -- 94.0%
nl: 177 correct out of 200 -- 88.5%


__Bi-gram models__

In [79]:
bi_lms = trainCharLM(1)
assessLM(bi_lms, 1)

fr: 196 correct out of 200 -- 98.0%
de: 198 correct out of 200 -- 99.0%
da: 198 correct out of 200 -- 99.0%
en: 200 correct out of 200 -- 100.0%
it: 200 correct out of 200 -- 100.0%
nl: 198 correct out of 200 -- 99.0%


__4-gram models__

In [80]:
four_lms = trainCharLM(3)
assessLM(four_lms, 3)

fr: 199 correct out of 200 -- 99.5%
de: 199 correct out of 200 -- 99.5%
da: 198 correct out of 200 -- 99.0%
en: 200 correct out of 200 -- 100.0%
it: 200 correct out of 200 -- 100.0%
nl: 199 correct out of 200 -- 99.5%
